In [3]:
import pandas as pd
from fuzzywuzzy import fuzz;
from fuzzywuzzy import process
from db_conn.connection import *

In [4]:
# Create dataframes
df1 = pd.DataFrame({'ref': ['12 FAM 041.1', '12 FAM 041.2', '12 FAM 043.1', '12 FAM 043.2-1\xa0', '12 FAM 081 (a)', '5 FAH-7 H-142', '5 FAM 741']})
df2 = pd.read_sql('select * from fam_grouped', engine())

In [6]:
# df1 will represent the dirty values, and may have some cleaning functions applied to them
df1['ref'] = df1['ref'].str.replace(r'[^A-Za-z0-9]', '', regex=True)
df2['ref'] = df2['ref'].str.replace(r'[^A-Za-z0-9]', '', regex=True)

In [7]:
# match hand-written policy references with entries in the Foreign Affairs Manual/Handbook
def fuzzy_lookup(source_df, target_df, source_col, target_col) -> pd.DataFrame():
    '''
        source_df: dataframe with the column/strings to lookup
        target_df: dataframe where you want to find matching values

        returns: dataframe
    '''
    threshold = .9
    matches = []
    for source_val in source_df[source_col]:
        best_match = process.extractOne(
            source_val,
            target_df[target_col],
            scorer=fuzz.ratio
        )
        if best_match and best_match[1] >= threshold:
            matches.append((source_val, best_match[0], best_match[1]))
        else:
            matches.append((source_val, None, None))
    return pd.DataFrame(matches, columns=[source_col, f'{target_col}_match', 'similarity'])

fuzzy_lookup(df1, df2, 'ref', 'ref')

,ref,ref_match,similarity
0,12FAM0411,12 FAM 041.1,86
1,12FAM0412,12 FAM 041.2,86
2,12FAM0431,12 FAM 043.1,86
3,12FAM04321,12 FAM 043.2-1,83
4,12FAM081a,12 FAM 081,84
5,5FAH7H142,5 FAM 712,67
6,5FAM741,5 FAM 741,88


In [8]:
df2

,file_name,title,heading,body_text,ref
0,01FAM0010.html,"1 FAM 010 AUTHORITY, RESPONSIBILITY, AND ORGAN...",,\n\n\n\n\n \n \n\n\n\n\n \n \n\n\n\n\n \n \n\n...,
1,01FAM0010.html,"1 FAM 010 AUTHORITY, RESPONSIBILITY, AND ORGAN...",AUTHORITY FOR CONDUCTING FOREIGN RELAT...,\n The Constitution vests in the President ...,1FAM011
2,01FAM0010.html,"1 FAM 010 AUTHORITY, RESPONSIBILITY, AND ORGAN...",Scope,\n a. The functional statements or organiza...,1FAM0111
3,01FAM0010.html,"1 FAM 010 AUTHORITY, RESPONSIBILITY, AND ORGAN...",Role of Department of State,\n The Department of State exists to assist...,1FAM0112
4,01FAM0010.html,"1 FAM 010 AUTHORITY, RESPONSIBILITY, AND ORGAN...",THE SECRETARY OF STATE’S AUTHORITY,\n a. The Secretary of State’s basic author...,1FAM012
...,...,...,...,...,...
20641,20FAM100102.html,20 FAM 1001.2 ESSENTIAL CONCEPTS (DOCUMENT AND...,Introduction,\n Reserved.\n \n \n \n \n,20FAM100121
20642,20FAM100201.html,20 FAM 1002.1 ROLES AND RESPONSIBILITIES (DOCU...,Introduction,\n Reserved.\n \n \n Reserved.\n,20FAM100211
20643,20FAM100202.html,20 FAM 1002.2 COMPLIANCE STANDARDS (DOCUMENT A...,Introduction,\n Reserved.\n \n \n Reserved.\n,20FAM100221
20644,20FAM100203.html,20 FAM 1002.3 COMPLIANCE REVIEW (DOCUMENT AND ...,Introduction,\n Reserved.\n \n \n Reserved.\n,20FAM100231
